In [1]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from datetime import datetime
import glob
from scipy.stats import gaussian_kde
from sklearn import preprocessing
import ipywidgets
from ipywidgets import widgets, fixed, VBox, HBox, ToggleButton, interact, interactive, IntSlider, Select, FloatSlider, IntRangeSlider, FloatRangeSlider, Checkbox, Dropdown, SelectMultiple
from IPython.display import clear_output

#######  setting  #############################################################
#数値実験基本設定
DontCareID = 99
FuzzyTypeID = {9:"rectangle", 7:"trapezoid", 3:"triangle", 4:"gaussian", DontCareID:"DontCare"}
myPath = os.getcwd()
cmap = plt.get_cmap("tab10")

DatasetList = {
    'iris': {'Patterns': 150, 'Attribute':4, 'Class':3},
    'wine': {'Patterns': 178, 'Attribute':13, 'Class':3},
    'phoneme': {'Patterns': 5404, 'Attribute':5, 'Class':2},
    'yeast': {'Patterns': 1484, 'Attribute':8, 'Class':10},
    'sonar': {'Patterns': 208, 'Attribute':60, 'Class':2},
    'pima': {'Patterns': 760, 'Attribute':8, 'Class':2},
    'vehicle': {'Patterns': 946, 'Attribute':18, 'Class':4},
    'bupa': {'Patterns': 345, 'Attribute':6, 'Class':2},
    'satimage': {'Patterns': 6435, 'Attribute':36, 'Class':6},
    'bal': {'Patterns': 630, 'Attribute':4, 'Class':3},
    'australian': {'Patterns': 690, 'Attribute':14, 'Class':2}
}

#scatterの基本設定
MARKER_SIZE = 100
MARKER_ALPHA = 0.01
#figureの基本設定
PLOT_TITLE_FLAG = False
PLOT_TICK_FLAG = False
PLOT_LABEL_FLAG = False
PLOT_GRID_FLAG = True
DEFAULT_FIG_SIZE = (16, 9) #画像のサイズ
DEFAULT_TITLE_SIZE = 30 #タイトルのフォントサイズ
DEFAULT_LABEL_SIZE = 30 #ラベルのフォントサイズ
DEFAULT_TICK_SIZE = 20 #目盛りのフォントサイズ
DEFAULT_LEGEND_SIZE = 20 #汎用のフォントサイズ
MARGIN_SIZE = {'left':0.06, 'right':0.94, 'bottom':0.06, 'top':0.94} #余白サイズ
IS_TRANSPARENT = False #背景の透過
#その他の基本設定
IMAGE_SAVE_DIR_PATH = "results/image/"
ONLY_ALL_GENERATION_FLAG = True
###############################################################################

In [2]:
### READING XML FILE SUPER CLASS ####################################################
class XML:
    """xmlファイルを読み込むためのスーパークラス"""
    def __init__(self, filename, show = True):
        self.tree = ET.parse(filename)
        self.rootNode = self.tree.getroot()
        if show : self.nodelist(self.rootNode)
        self.CurrentElement = [] #木構造の現在参照しているノードの位置を保存する

    def nodelist(self, root):
        c = True
        for i, child in enumerate(root):
            if i<5 or i>len(list(root))-3:
                txt = "node" if child.text == None else str(child.text)
                print("{:3d}: {:20}{:10}{}".format(i, child.tag, str(txt), child.attrib))
            else:
                if c:
                    print("         .\n         .\n         .\n")
                    c = False
        
    def down(self, id):
        """指定したIDの子要素を参照する"""
        self.CurrentElement.append(id)
        buf = self.rootNode
        for i in self.CurrentElement:
            if not list(buf):
                print("NULL")
                self.up()
                return
            buf = buf[i]
        print("current node:" + buf.tag)
        print(buf.attrib)
        print("\n")
        self.nodelist(buf)
            
    def up(self):
        """指定したIDの親要素を参照する"""
        self.CurrentElement.pop(-1)
        buf = self.rootNode
        for i in self.CurrentElement:
            buf = buf[i]
        print("current node:" + buf.tag)
        print(buf.attrib)
        print("\n")
        self.nodelist(buf)
            
    def root(self):
        """木の根にもどる"""
        self.CurrentElement.clear()
        self.nodelist(self.rootNode)
        
    def showAll(self):
        buf = self.rootNode
        for i in self.CurrentElement:
            buf = buf[i]
        for i, child in enumerate(buf):
            txt = "node" if child.text == None else str(child.text)
            print("{:3d}: {:20}{:10}{}".format(i, child.tag, str(txt), child.attrib))
            
    def show(self):
        buf = self.rootNode
        for i in self.CurrentElement:
            buf = buf[i]
        for i, child in enumerate(buf):
            if i<5 or i>len(list(buf))-3:
                txt = "node" if child.text == None else str(child.text)
                print("{:3d}: {:20}{:10}{}".format(i, child.tag, str(txt), child.attrib))
            elif i>9 and i<13:
                print("         .")
###############################################################################      

In [3]:
### FIGURE OBJECT FUNCTIONS ###################################################       
# SETTING SINGLE FIGURE OBJECT
def singleFig_set(title = None):
    """保存する画像(グラフ1つ)の基本設定
    入力:ファイル名
    返り値:figureオブジェクト"""
    fig = plt.figure(figsize = DEFAULT_FIG_SIZE)
    plt.rcParams["font.family"] = "MS Gothic"
    fig.subplots_adjust(left=MARGIN_SIZE['left'], right=MARGIN_SIZE['right'], bottom=MARGIN_SIZE['bottom'], top=MARGIN_SIZE['top'])
    fig.add_subplot(1, 1, 1)
    if title is not None: fig.suptitle(title, size = DEFAULT_TITLE_SIZE)        
    return fig

# SETTING MULTI FIGURE OBJECT    
def multiFig_set(x_FigNum, y_FigNum,  title = None):
    """保存する画像(グラフ複数)の基本設定
    入力:ファイル名 axNum:グラフの数
    返り値:figureオブジェクト"""
    fig = plt.figure(figsize = (x_FigNum*DEFAULT_FIG_SIZE[0], y_FigNum*DEFAULT_FIG_SIZE[1]))
    plt.rcParams["font.family"] = "MS Gothic"
    fig.subplots_adjust(left=MARGIN_SIZE['left'], right=MARGIN_SIZE['right'], bottom=MARGIN_SIZE['bottom'], top=MARGIN_SIZE['top'])
    if title is not None: fig.suptitle(title, size = DEFAULT_TITLE_SIZE*x_FigNum/2)
    return fig        
    
# OUTPUT FIGURE OBJECT
def saveFig(fig, filePath, filename = None):
    """画像を保存する
    入力:figureオブジェクト, ファイル名, データセット名"""
    if filename == None:
        print("image file name:")
        filename = input()
    os.makedirs(filePath, exist_ok=True)
    fig.savefig(filePath + "/" + filename, transparent=IS_TRANSPARENT) 
    
def plotRuleSetting(ax):
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    if PLOT_TICK_FLAG:
        ax.tick_params(axis="x", labelsize=DEFAULT_TICK_SIZE)
        ax.tick_params(axis="y", labelsize=DEFAULT_TICK_SIZE)
    else:
        ax.tick_params(labelbottom=False, labelleft=False)
    if PLOT_LABEL_FLAG:
        ax.set_xlabel("属性値", fontsize=DEFAULT_LABEL_SIZE,  fontname="MS Gothic")
        ax.set_ylabel("メンバシップ値", fontsize=DEFAULT_LABEL_SIZE,  fontname="MS Gothic")
    if PLOT_GRID_FLAG:
        ax.grid(True)
        ax.legend().get_frame().set_alpha(1.0)
        
def plotResultSetting(ax):
    ax.tick_params(axis="x", labelsize=DEFAULT_TICK_SIZE)
    ax.tick_params(axis="y", labelsize=DEFAULT_TICK_SIZE)
    ax.set_xlabel("ルール数", fontsize=DEFAULT_LABEL_SIZE,  fontname="MS Gothic")
    ax.set_ylabel("誤識別率[%]", fontsize=DEFAULT_LABEL_SIZE,  fontname="MS Gothic")
    plt.legend(fontsize = DEFAULT_LEGEND_SIZE)
    ax.grid(True)
###############################################################################

In [4]:
### XML FILE READER ################################################################


### Costs ###########################################################  
class Costs:
    """Costs.propertyのためのクラス"""
    def __init__(self, constsNode):
        self.settingParameters = {}
        for child in constsNode:
            if child.text != None:
                self.settingParameters[child.tag] = child.text
            else:
                self.settingParameters[child.tag] = [int(grandChild.text) for grandChild in child]
                
    def getParameter(self, ParameterName):
        return self.settingParameters[ParameterName]
    
    
### Fuzzy Term ###########################################################                  
class FuzzyTerm:
    """Fuzzy Termのためのクラス"""
    def __init__(self, fuzzyTerm):
        self.fuzzyTermID = int(fuzzyTerm.find("fuzzyTermID").text)
        self.fuzzyTermName = fuzzyTerm.find("fuzzyTermName").text
        self.ShapeTypeID = int(fuzzyTerm.find("ShapeTypeID").text)
        self.rectangularShape = fuzzyTerm.find("ShapeTypeName").text
        self.parameters = {int(parameters_i.get('dimentionID')) : float(parameters_i.text) for parameters_i in fuzzyTerm.find('parameters')}
    
    def setAx(self, ax, alpha = 1.0, alpha_between = 0.3, color = "black", color_between = "blue", input_x=None):
        """Ax にメンバシップ関数をプロットする"""
        color_buf = "C{}".format(color) if type(color) is int else color
        
        ax.set_ylim(-0.05, 1.05)
        if not input_x == None:
            y_data = self.membershipValue(input_x)
            ax.vlines(x=input_x, ymin=0, ymax=y_data, colors="black")
            ax.hlines(y=y_data, xmin=0, xmax=input_x, colors="black")
            
        if self.ShapeTypeID == 3:
            x = np.array([0, self.parameters[0], self.parameters[1], self.parameters[2], 1])
            y = np.array([0, 0, 1, 0, 0])
            ax.plot(x, y, alpha = alpha, color = color_buf)
            y_bottom = [0, 0, 0, 0, 0]
            ax.fill_between(x, y, y_bottom, facecolor=color_between, alpha = alpha_between)
        if self.ShapeTypeID == 4:
            mu = self.parameters[0]
            sigma = self.parameters[1]
            x = np.arange(0, 1, 0.01)
            y = 1 * np.exp(-(x - mu)**2 / (2*sigma**2))
            ax.plot(x, y, alpha = alpha, color = color_buf)
            y_bottom = [0] * 100
            ax.fill_between(x, y, y_bottom, facecolor = color_between, alpha = alpha_between)
        if self.ShapeTypeID == 7:
            x = np.array([0, self.parameters[0], self.parameters[1], self.parameters[2], self.parameters[3], 1])
            y = np.array([0, 0, 1, 1, 0, 0])
            ax.plot(x, y, alpha = alpha, color = color_buf)
            y_bottom = [0, 0, 0, 0, 0, 0]
            ax.fill_between(x, y, y_bottom, facecolor = color_between, alpha = alpha_between)
        if self.ShapeTypeID == 9:
            if self.parameters[0] < 0: x = np.array([0, 0, 0, self.parameters[1], self.parameters[1], 1])
            else: x = np.array([0, self.parameters[0], self.parameters[0], self.parameters[1], self.parameters[1], 1])
            y = np.array([0, 0, 1, 1, 0, 0])
            ax.plot(x, y, alpha = alpha, color = color_buf)
            y_bottom = [0, 0, 0, 0, 0, 0]
            ax.fill_between(x, y, y_bottom, facecolor = color_between, alpha = alpha_between)
        if self.ShapeTypeID == 99:
            x = np.array([0, 0, 1, 1])
            y = np.array([0, 1, 1, 0])
            ax.plot(x, y, alpha = alpha, color = color_buf)
            y_bottom = [0, 0, 0, 0]
            ax.fill_between(x, y, y_bottom, facecolor = color_between, alpha = alpha_between)
            
    def membershipValue(self, input_x):
        """メンバシップ値を出力する"""
        
        if self.ShapeTypeID == 3:
            x_1, x_2, x_3 = self.parameters[0], self.parameters[1], self.parameters[2]
            if input_x <= x_1: return 0
            elif x_1 < input_x and input_x <= x_2: return (input_x - x_1)/(x_2 - x_1)
            elif x_2 < input_x and input_x <= x_3: return (x_3 - input_x)/(x_3 - x_2)
            elif x_3 < input_x: return 0
        if self.ShapeTypeID == 4:
            mu = self.parameters[0]
            sigma = self.parameters[1]
            return  1 * np.exp(-(input_x - mu)**2 / (2*sigma**2))
        if self.ShapeTypeID == 7:
            x_1, x_2, x_3, x_4 = self.parameters[0], self.parameters[1], self.parameters[2], self.parameters[3]
            if input_x <= x_1: return 0
            elif x_1 < input_x and input_x <= x_2: return (input_x - x_1)/(x_2 - x_1)
            elif x_2 < input_x and input_x <= x_3: return 1
            elif x_3 < input_x and input_x <= x_4: return (input_x - x_3)/(x_4 - x_3)
            elif x_4 < input_x: return 0
        if self.ShapeTypeID == 9:
            x_1, x_2 = self.parameters[0], self.parameters[1]
            if input_x <= x_1: return 0
            elif x_1 < input_x and input_x <= x_2: return 1
            elif x_2 < input_x: return 0
        if self.ShapeTypeID == 99:
            return 1
        
        
### Knowledge Base ###########################################################          
class KnowledgeBase:
    """Knowledge bas用のクラス"""
    def __init__(self, knowledge):
        self.fuzzySets = {}
        for fuzzySets in knowledge.findall('fuzzySets'):
            self.fuzzySets[int(fuzzySets.get("dimentionID"))] = {int(fuzzyTerm.find("fuzzyTermID").text) : FuzzyTerm(fuzzyTerm) for i, fuzzyTerm in enumerate(fuzzySets)}
            
    def membershipValue(self, dimentionID, FuzzyTermID, input_x):
        return self.fuzzySets[dimentionID][FuzzyTermID].membershipValue(input_x)
    
    def setAx(self, ax, dimentionID, fuzzyTermID, color_between="blue", input_x=None):
        self.fuzzySets[dimentionID][fuzzyTermID].setAx(ax, color_between=color_between, input_x=input_x)
        
        
### Single Rule ###########################################################  
class SingleRule():
    def __init__(self, singleRule):
        #antecedent
        self.fuzzyTermIDVector = {int(fuzzyTermID.get('dimentionID')) : int(fuzzyTermID.text) for fuzzyTermID in singleRule.find('antecedent').findall('fuzzyTermID')}
        
        #consequent
        self.consequentClassesVector = {int(consequentClasses.get('ClassID')) : int(consequentClasses.text) for consequentClasses in singleRule.find('consequent').find('consequentClasses')}
        self.ruleWeightsVector = {int(ruleWeights.get('ruleWeightID')) : float(ruleWeights.text) for ruleWeights in singleRule.find('consequent').find('ruleWeights')}
        
    def getFuzzyTermIDList(self):
        return list(self.fuzzyTermIDVector.values())
    
    def getFuzzyTermID(self, dimID):
        return self.fuzzyTermIDVector[dimID]

### Classifier ###########################################################  


class Classifier():
    def __init__(self, classifier):
        self.singleRules = {rule_i : SingleRule(singleRule) for rule_i, singleRule in enumerate(classifier.findall('singleRule'))}

### Individual ###########################################################  
class Individual():
    """pittsbugh型用個体"""
    def __init__(self, individual, knowledge, basicData):
        self.knowledge = knowledge
        self.basicData = basicData
        self.classifier = Classifier(individual.find('classifier'))
        self.accuracyRate_Dtra = float(individual.find("accuracyRate_Dtra").text)
        self.accuracyRate_Dtst = float(individual.find("accuracyRate_Dtst").text)/100
        self.ruleNum = int(float(individual.find("ruleNum").text))
        
    def getSingleRule(self, ruleID):
        return self.classifier.singleRules[ruleID]
    
    def getSingleRules(self):
        return self.classifier.singleRules
    
    def isSingleRuleCoverAllClasses(self, ClassNum):
        if self.ruleNum < ClassNum:
            return False
        else :
            buf = set()
            for singleRule in self.classifier.singleRules.values():
                if not singleRule.consequentClassesVector[0] in buf : buf.add(singleRule.consequentClassesVector[0])
            if len(buf) == ClassNum: return True
            else: return False
    
    def getClassifierDict(self):
        return {ruleID: singleRule.getFuzzyTermIDList() for ruleID, singleRule in self.classifier.singleRules.items()} 
    
    def plotRule(self, ruleIDs, coloring='class'):
        """指定したID(int or list型)のruleをプロット"""
        if coloring == 'each rules': color_between=cmap(rule_i)
        elif coloring == 'class': color_between=cmap(singleRules.consequentClassesVector[0])
        elif coloring == 'single': color_between = cmap(0)
            
        fig = multiFig_set(y_FigNum = 1, x_FigNum=self.basicData['attributeNum'], title='Rule: ' + str(ruleID))
        ruleID_list = [ruleIDs] if type(ruleIDs) is not int else ruleIDs
        cnt = 1
        for rule_i in enumerate(ruleID_list):
            singleRule = self.getSingleRule(rule_i)
            for attribute_i in self.basicData['attributeNum']:
                ax = fig.add_subplot(len(ruleID_list), self.basicData['attributeNum'], cnt)
                cnt += 1
                self.knowledge.setAx(ax, attribute_i, singleRule.getFuzzyTermID(AttributeID), color_between=color_between)
                plotRuleSetting(ax)
    
    def plotRuleMulti(self, coloring='class', inputPattern = None):
        """１つの画像に全てのif-thenルールをプロット"""
        fig = multiFig_set(y_FigNum = self.ruleNum, x_FigNum=self.basicData['attributeNum'], title='Rule')
        cnt = 1
        for rule_i in range(self.ruleNum):
            for attributeID in range(self.basicData['attributeNum']):
                ax = fig.add_subplot(self.ruleNum, self.basicData['attributeNum'], cnt)
                cnt += 1
                if coloring == 'each rules': color_between=cmap(rule_i)
                elif coloring == 'class': color_between=cmap(self.getSingleRule(rule_i).consequentClassesVector[0])
                elif coloring == 'single': color_between = cmap(0)
                if inputPattern is not None:
                    self.knowledge.setAx(ax, attributeID, self.getSingleRule(rule_i).getFuzzyTermID(attributeID), color_between=color_between, input_x=inputPattern[attributeID])
                else:
                    self.knowledge.setAx(ax, attributeID, self.getSingleRule(rule_i).getFuzzyTermID(attributeID), color_between=color_between)
                plotRuleSetting(ax)
        cnt = 1
        plt.show()
        
### Population ###########################################################  
class Population():
    """個体群"""
    def __init__(self, population, knowledge, basicData):
        self.knowledge = knowledge
        self.individuals = {i: Individual(individual, knowledge, basicData) for i, individual in enumerate(population.findall('individual'))}
        self.evaluationID = int(population.get('evaluations'))
        if 'individualNum' not in basicData : basicData['individualNum'] = len(self.individuals)
        self.basicData = basicData
    
    def getIndividual(self, individualID):
        return self.individuals[individualID]
    
    def getSingleRule(self, individualID, ruleID):
        return self.individuals[individualID].getSingleRule(ruleID)
    
    def getSingleRules(self, individualID):
        return self.individuals[individualID].getSingleRules()
    
    def getIndividuals_All(self, ruleBook, dataList = [], plotData = 'isDtra'):
        """全個体をlist型で出力"""
        for individual in self.individuals.values():
            if not ('option1' in ruleBook and individual.ruleNum < 2):
                if plotData == 'isDtra': dataList.append((individual.ruleNum, individual.accuracyRate_Dtra))
                else : dataList.append((individual.ruleNum, individual.accuracyRate_Dtst))
        return dataList
    
    def getIndividualsAverage(self, ruleBook, plotData='isDtra'):
        """各個体の平均値をlist型で出力"""
        dataBuf = {}
        for individual in self.individuals.values():
            if ('option1' in ruleBook and individual.ruleNum > 1 ) or 'option1' not in ruleBook:
                if ('option3' in ruleBook and individual.isSingleRuleCoverAllClasses(self.basicData['classNum'])) or 'option3' not in ruleBook:
                    if individual.ruleNum in dataBuf:
                        if plotData == 'isDtra': dataBuf[individual.ruleNum]['sum'] += individual.accuracyRate_Dtra
                        else : dataBuf[individual.ruleNum]['sum'] += individual.accuracyRate_Dtst
                        dataBuf[individual.ruleNum]['num'] += 1
                    else:
                        if plotData == 'isDtra': dataBuf[individual.ruleNum] = {'sum':individual.accuracyRate_Dtra, 'num':1}
                        else : dataBuf[individual.ruleNum] = {'sum':individual.accuracyRate_Dtst, 'num':1}
        individuals = [(key, value['sum']/value['num']) for key, value in dataBuf.items()]
        return individuals
    
    def clacErrorRate(self, individualID, data):
        """誤識別率を出力"""
        buf = {}
        for ruleID, singleRule in self.getSingleRules(individualID).items():
            tmp = 1
            print('Rule ' + str(ruleID) + ': ', end='')
            for dimID, FuzzyTermID in enumerate(singleRule.getFuzzyTermIDList()):
                tmp2 = self.knowledge.membershipValue(dimID, FuzzyTermID, data[dimID])
                tmp *= tmp2
                print( 'x_{} is {:3f}'.format(dimID, tmp2), end=' ')
            tmp *= singleRule.ruleWeightsVector[0]
            buf[ruleID] = (tmp, singleRule.consequentClassesVector[0])
            print('then (Class{}, {:5f}) with {:5f}'.format(singleRule.consequentClassesVector[0], tmp, singleRule.ruleWeightsVector[0]))
        return buf    
    
### Trial Manager ###########################################################

class TrialManager(XML):
    def __init__(self, path, savePath, basicData):
        """一試行用のクラス
        入力: path=xmlファイルのパス, savePath 保存先指定フォルダ"""
        self.savePath = savePath
        os.makedirs(self.savePath, exist_ok=True)
        super().__init__(path, show = False)
        knowledge = KnowledgeBase(self.rootNode.find('knowledge'))
        #self.consts = Costs(self.rootNode.find('consts'))
        if ONLY_ALL_GENERATION_FLAG:
            self.populations = {int(population.get('evaluations')) : Population(population, knowledge, basicData) for population in self.rootNode.findall('population')}
        else:
            population = self.rootNode.findall('population')
            print(population)
            self.populations = {int(population.get('evaluations')) : Population(population, knowledge, basicData)}
        if 'evaluationList' not in basicData : basicData['evaluationList']=list(self.populations.keys())
        self.basicData = basicData
        
    def getPopulation(self, evaluation = None):
        if evaluation is None : evaluation = int(self.consts.settingParameters['terminateEvaluation'])
        return self.populations[evaluation]
    
    def getKnowledge(self):
        return self.knowledge

### Experiment Manager ###########################################################
class ExperimentManager():
    def __init__(self, path, savePath, label, basicData):
        """一つの実験用のクラス
        入力: path=xmlファイルのパス, savePath 保存先指定フォルダ"""
        self.label = label
        self.savePath = savePath
        os.makedirs(self.savePath, exist_ok=True)
        self.TrialManagers = {}
        XMLpaths = [str.replace(os.path.sep,"/") for str in glob.glob(path + "/trial*/*.xml")]
        for XMLpath in XMLpaths:
            self.TrialManagers[XMLpath.split('/')[-2]] = TrialManager(XMLpath, savePath + "/" + XMLpath.split('/')[-2], basicData)
        if 'trialNum' not in basicData : basicData['trialNum']=len(XMLpaths)
        self.basicData = basicData
        
    def getTrial(self, trialID):
        return self.TrialManagers['trial{:0>2}'.format(trialID)]
    
    def getPopulation(self, trialID, evaluation):
        return self.TrialManagers['trial{:0>2}'.format(trialID)].getPopulation(evaluation)
    
    def getIndividuals_Average(self, evaluation, ruleBook, plotData='isDtra'):
        """全試行で得られた全個体の平均値を出力"""
        individualsBuf = {}
        for trial in self.TrialManagers.values():
            dataList = trial.getPopulation(evaluation).getIndividualsAverage(ruleBook=ruleBook, plotData=plotData)
            for data in dataList:
                if data[0] in individualsBuf:
                    individualsBuf[data[0]]['sum'] += data[1]
                    individualsBuf[data[0]]['num'] += 1
                else:
                    individualsBuf[data[0]] = {'sum':data[1], 'num':1}
                    
        if 'option2' in ruleBook:
            individuals = [(key, value['sum']/value['num']) for key, value in individualsBuf.items() if value['num'] > self.basicData['trialNum']/2]
        else:
            individuals = [(key, value['sum']/value['num']) for key, value in individualsBuf.items()]
            
        return individuals
    
    def getIndividuals_AverageAllTrial(self, evaluation, ruleBook, plotData='isDtra'):
        """1回試行で得られた個体群の平均値を出力"""
        individuals = []
        for trial in self.TrialManagers.values():
            individuals.extend(trial.getPopulation(evaluation).getIndividualsAverage(ruleBook=ruleBook, plotData=plotData))
        return individuals

    def getIndividuals_All(self, evaluation, ruleBook, plotData='isDtra'):
        """全個体を出力"""
        buf = []
        for trial in self.TrialManagers.values():
            population = trial.getPopulation(evaluation)
            population.getIndividuals_All(dataList = buf, plotData=plotData, ruleBook=ruleBook)
        return buf        
    
###############################################################################

In [5]:
### PLOT XML FILE #############################################################

class Master:
    def __init__(self, dataSetName, currentDir, ExperimentList, ExperimentLabel):
        self.dataSetName = dataSetName
        self.currentDir = currentDir
        self.ALGORITHM_ID_DIR = [self.currentDir + "/results/" + Experiment_i for Experiment_i in ExperimentList]
        print(self.ALGORITHM_ID_DIR)
        self.DATA_LABEL = ExperimentLabel
        self.basicData = {'datasetName':self.dataSetName, 'classNum':DatasetList[self.dataSetName]['Class'], 'attributeNum':DatasetList[self.dataSetName]['Attribute']}
        self.master = {}
        for i in range(len(self.ALGORITHM_ID_DIR)):
            self.master[self.DATA_LABEL[i]] = ExperimentManager(self.ALGORITHM_ID_DIR[i] + "/" + self.dataSetName, self.ALGORITHM_ID_DIR[i] + "/" + self.dataSetName + "/plot", self.DATA_LABEL[i], self.basicData)
    
    def getTrialManager(self, dataLabel, trialID):
        return self.master[dataLabel].TrialManagers['trial{:0>2}'.format(trialID)]
    
    def getPopulation(self, dataLabel, trialID, evaluationID):
        return self.master[dataLabel].TrialManagers['trial{:0>2}'.format(trialID)].getPopulation(evaluationID)
    
    def getIndividual(self, dataLabel, trialID, evaluationID, individualID):
        return self.master[dataLabel].TrialManagers['trial{:0>2}'.format(trialID)].getPopulation(evaluationID).individuals[individualID]
    
    def plotResult(self, model, evaluation, mode, xlim, ylim, alpha, ruleBook, plotData='isDtra', saveFig_c=False):
        """学習結果をプロット．
        model: ExperimentManager識別用, evaluation: 世代数, mode: プロットするモード選択, xlim: x軸のレンジ, ylim: y軸のレンジ, 
        alpha: ドットの透明度, ruleBook: 表示個体の制限, plotData: 評価用データor学習用データ, saveFig_c: 画像を保存するか否か"""

        if mode=='all':
            fig = singleFig_set('all_eva' + str(evaluation))
            fig.subplots_adjust(bottom=0.12)
            ax = fig.gca()
            for experimentLabel in model:
                experimentManager = self.master[experimentLabel]
                data = experimentManager.getIndividuals_All(evaluation=evaluation, plotData=plotData, ruleBook=ruleBook)
                ax.scatter([data_i[0] for data_i in data], [data_i[1] for data_i in data], s=MARKER_SIZE, alpha=alpha, label=experimentManager.label)

        elif mode=='average_AllTrial':
            fig = singleFig_set('average_eva' + str(evaluation))
            fig.subplots_adjust(bottom=0.12)
            ax = fig.gca()        
            for experimentLabel in model:
                experimentManager = self.master[experimentLabel]
                data = experimentManager.getIndividuals_AverageAllTrial(evaluation=evaluation, ruleBook=ruleBook, plotData=plotData)
                ax.scatter([data_i[0] for data_i in data], [data_i[1] for data_i in data], s=MARKER_SIZE, alpha=alpha, label=experimentManager.label)                
        
        elif mode=='average':
            fig = singleFig_set('average_eva' + str(evaluation))
            fig.subplots_adjust(bottom=0.12)
            ax = fig.gca()        
            for experimentLabel in model:
                experimentManager = self.master[experimentLabel]
                data = experimentManager.getIndividuals_Average(evaluation=evaluation, ruleBook=ruleBook, plotData=plotData)
                ax.scatter([data_i[0] for data_i in data], [data_i[1] for data_i in data], s=MARKER_SIZE, alpha=alpha, label=experimentManager.label, color = "black")
                
        ax.set_xlim([xlim[0], xlim[1]])
        ax.set_ylim([ylim[0], ylim[1]])
        ax.set_xticks(range(int(xlim[1])+1))
        plotResultSetting(ax)
        
        if saveFig:
            dirName = "_".join(model)
            dataType = "Dtra" if plotData == 'isDtra' else "Dtst"
            fileName = "result_{:s}_{:s}_{:s}_{:s}_{:08}".format(self.dataSetName, dataType, dirName, mode, evaluation)
            saveFig(fig, self.currentDir + "/" + IMAGE_SAVE_DIR_PATH + self.dataSetName + "/" + dirName + "/" + dataType, fileName)
        plt.show()
        
    def plotResultInteract(self):
        """学習結果用．タラクティブな処理のためのインターフェイス"""
        evaluationIDList = list(self.master[self.DATA_LABEL[0]].getTrial(0).populations.keys())
        evaluationIDList.sort()
        evaluationID = IntSlider(value=evaluationIDList[-1], min=evaluationIDList[0], max=evaluationIDList[-1], step=evaluationIDList[1]-evaluationIDList[0])
        play = widgets.Play(
            value=evaluationIDList[-1], min=evaluationIDList[0], max=evaluationIDList[-1],
            step=evaluationIDList[1]-evaluationIDList[0], interval=1000,
            description="Press play", disabled=False
        )
        widgets.jslink((play, 'value'), (evaluationID, 'value'))
        saveFig = widgets.Checkbox(value=False, description='save figures')

        xlim = IntRangeSlider(value=[0, 20], min=0, max=50, description = "X lim:")
        ylim = FloatRangeSlider(value=[0, 1.0], min=0, max=1.05, step=0.01, description='Y lim:')
        alpha = FloatSlider(value=0.3, min=0, max=1, step=0.01, description='alpha:')
        model = SelectMultiple(options=self.DATA_LABEL, value=self.DATA_LABEL, description='models:')
        
        mode = Select(description = "mode", options = ["all", "average_AllTrial", "average"], rows = 3)
        options = SelectMultiple(options=['option1', 'option2', 'option3'], value=['option1', 'option2', 'option3'], description='option:')
        rule1_escript = widgets.Label('option1: Rules more than 2 individuals')
        rule2_escript = widgets.Label('option2: Populations more than half of the number of trial')
        rule3_escript = widgets.Label('option3: Rules cover all classes')
        plotData = widgets.ToggleButtons(options=['isDtra', 'isDtst'], description='data:', button_style='', tooltips=['train data', 'tst data'])
        
        accordion = widgets.Accordion(children=[VBox([HBox([xlim, ylim, alpha]),model]), VBox([mode, options, rule1_escript, rule2_escript, rule3_escript])])
        accordion._titles = {0:'graph parameters', 1:'options'}
        graph = widgets.interactive_output(self.plotResult, {'model':model, 'evaluation':evaluationID, 'mode':mode, 'xlim':xlim, 'ylim':ylim, \
                                                         'alpha':alpha, 'ruleBook':options, 'plotData':plotData, 'saveFig_c':saveFig})
        display(HBox([plotData, play, evaluationID, saveFig]))
        display(accordion)
        display(graph)
                                               
    def plotRuleInteract(self):
        """ルール表示用．タラクティブな処理のためのインターフェイス"""
        dataLabel = widgets.Dropdown(options=self.DATA_LABEL, value=self.DATA_LABEL[0], description='DATA_LABEL')
        evaluationIDList = self.basicData['evaluationList']
        evaluationIDList.sort()
        evaluationID = IntSlider(value=evaluationIDList[-1], min=evaluationIDList[0], max=evaluationIDList[-1], step=evaluationIDList[1]-evaluationIDList[0], description = "evaluation: ")
        trialID = IntSlider(value=0, min=0, max=self.basicData['trialNum']-1, description = "Trial: ")
        individualID = IntSlider(value=0, min=0, max=self.basicData['individualNum']-1, description = "Individual: ")
        
        coloring = Dropdown(options=['single', 'each rules', 'class'], value='class', description='color system: ')
        
        display(HBox([VBox([dataLabel, coloring]), VBox([evaluationID, trialID, individualID])]))
        
        title = {i:'attribute_' + str(i) for i in range(self.basicData['attributeNum'])}
        
        rulePlot = widgets.interactive_output(self.plotRule, {'dataLabel':dataLabel, 'evaluationID':evaluationID, 'trialID':trialID, 'individualID':individualID, 'coloring':coloring, })
        display(rulePlot)
        
    def plotRule(self, dataLabel, evaluationID, trialID, individualID, coloring='class'):
        """ルールを表示
        dataLabel: ExperimentManager識別用, evaluationID: 世代数, trialID: 試行番号, individualID: 個体番号, coloring: カラーリングのルール"""
        population = self.getPopulation(dataLabel=dataLabel, evaluationID=evaluationID, trialID=trialID)
        individual = population.getIndividual(individualID)
        widgets.interact_manual(individual.plotRuleMulti, coloring=ipywidgets.fixed(coloring), inputPattern=ipywidgets.fixed(None))
        
###############################################################################

In [6]:
#データセットの名前
DATA_SET_NAME = "pima"

#MoFGBMLのフォルダが置いてあるディレクトリ
CURRENT_DIR_PATH = "E:/OPU/lab/MoFGBML_renew"

#results以下の各実験フォルダ名
EXPERIMENT_LIST = ["test", "default"]

#グラフの凡例において，各実験に付けたいラベル名
EXPERIMENT_LABEL = ["renew", "before"]

master = Master(DATA_SET_NAME, CURRENT_DIR_PATH, EXPERIMENT_LIST, EXPERIMENT_LABEL)

In [8]:
master.plotResultInteract()
%matplotlib inline

Accordion(children=(VBox(children=(HBox(children=(IntRangeSlider(value=(0, 20), description='X lim:', max=50),…

Output()

In [9]:
master.plotRuleInteract()
%matplotlib inline

Output()

In [10]:
from PIL import Image
def makeGIF(folderName, dataSetType):
    folderPath = CURRENT_DIR_PATH + "/" + IMAGE_SAVE_DIR_PATH + master.dataSetName + "/" + folderName + "/" + dataSetType
    filename = "result_" + master.dataSetName + "_" + dataSetType
    files = sorted(glob.glob(folderPath + "/" + filename + '*.png'))
    images = list(map(lambda file: Image.open(file), files))
    os.makedirs(folderPath, exist_ok=True)
    images[0].save(folderPath + '/' + filename + '.gif', save_all=True, append_images=images[1:], duration=400, loop=0)


In [11]:
FOLDER_NAME = "renew_before"
DATA_SET_TYPE =  "Dtra"
#makeGIF(FOLDER_NAME, DATA_SET_TYPE)